In [1]:
#load the csv to the notebook
import pandas as pd
import datetime 

In [2]:
rawInput=pd.read_csv('lasvegasStrip.csv',encoding="ISO-8859-1")
rawInput.head()

,Unnamed: 0,user,location,date,rating,title,comments
0,0,s6steve,"Sheffield, United Kingdom","July 9, 2018",20,Over priced over rated,Over rated new Las Vegas has taken away the sp...
1,0,sathyanupheap,unknown,"July 9, 2018",40,Plenty to See...,I rented a Mini Cooper convertible and it was ...
2,0,Blu H,unknown,"July 9, 2018",50,City of anything goes!,If you haven¡¯t been here yet you¡¯re probably...
3,0,anita a,"Los Angeles, California","July 9, 2018",30,Not like it use to be,Back in the day you could eat and have fun on ...
4,0,kristycullinan,"Philadelphia, Pennsylvania","July 9, 2018",30,Walking walking and more walking.,The strip is 5 miles long and pretty spaced ou...


In [3]:
###put a time range for it
#rawInput['realDate']=[datetime.datetime.strptime(i,'%B %d, %Y') for i in rawInput['date']]
#rawInput=rawInput[rawInput['realDate']>datetime.datetime.strptime('January 01, 2014','%B %d, %Y')]

## preprocess text

In [4]:
#import the nltk lib for preprocessing
import nltk
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer

In [5]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [6]:
#nltk.download() #make sure the computer has all nltk corpus

In [7]:
import string #for getting punctuation
import gensim

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [8]:
stop = set(stopwords.words('english'))
excludeMyself=['put','thats','that','what','whats','vega','la','las','thing','soooo','so','soo','sooo','world','si','lot','ops','didnt','cant','some',
              'somehow','describe','name','whilst','even','actual','one','two','three','four','five','six','seven','eight','nine','ten','duck','amongst',
              'something','most','fully','what','whats','whatever','many','stip','strip','occasional','occasionally','wont','will','non','i','ive']
exclude = list(set(string.punctuation))
lemma = WordNetLemmatizer()##WordNetLemmatizer is a class

In [9]:
print (exclude)

['@', '_', '}', '%', '\\', '"', ':', '<', '~', '|', '{', '.', '[', '$', '/', '*', '-', '(', ',', '>', '+', "'", '`', '=', ';', '!', ']', '?', ')', '#', '^', '&']


In [10]:
##a def to clean the text(typical way), very nice
def clean(doc):
    stop_free = " ".join([i for i in gensim.utils.simple_preprocess(doc,min_len=3) if i not in stop])
    ##simple_preprocess can lowercase words and tokenize them,you can set up min len of token too,here is 3, so word like AI will not be included
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())##这个用来找stem
    myselfOutput = " ".join(word for word in normalized.split() if word not in excludeMyself)##这个用来找stem
    return myselfOutput

In [11]:
clean(rawInput['comments'][0])

'rated new taken away spark original prepared pant drink bother night club guy pretty sexist priced staff everywhere arrogant care long'

## here is the text dataset

In [12]:
doc_complete=rawInput['comments'][:]

In [13]:
doc_clean = [clean(doc).split() for doc in doc_complete]    
#doc_clean is a list中包list,每个item是一个doc的tokens

In [14]:
#doc_clean[:2]

## create gensim corpus dicts and doc_term_Matrix

In [15]:
# Importing Gensim
import gensim
from gensim import corpora

In [16]:
# Creating the term dictionary of our courpus, where every unique term（token) is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

In [17]:
print(len(dictionary))
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

16754
0 arrogant
1 away
2 bother
3 care
4 club
5 drink
6 everywhere
7 guy
8 long
9 new
10 night


In [18]:
#### filter_extremes可以把出现less than 15 documents的tokens (absolute number) 
#or more than 0.5 documents (fraction of total dictionary size, not absolute number). dictionary里有一半都是那个词删掉。
#after the above two steps, keep only the first 100000 most frequent tokens.
#dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000) #当文字很多的时候用这个,先不run

In [19]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean] #这个也叫做bow

In [20]:
print (doc_clean[0])

['rated', 'new', 'taken', 'away', 'spark', 'original', 'prepared', 'pant', 'drink', 'bother', 'night', 'club', 'guy', 'pretty', 'sexist', 'priced', 'staff', 'everywhere', 'arrogant', 'care', 'long']


In [21]:
print (doc_term_matrix[0])###这个matrix是每个doc的list of (token_id, token_count) tuples

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)]


In [22]:
print (dictionary[0])#dictionary 可以直接输入token的id得出token
dictionary[doc_term_matrix[0][2][0]]#找到第一个id为0的词

arrogant


'bother'

## tf-idf model LDA
create tf-idf scores, 这个score就是给每个token一个score，这score的好处是：当很多doc都有某个词比如city，那么city对选择topic意义并不大，当有些词是某个doc独有如accessible，那么这个词的意义还更强。。
https://rare-technologies.com/pivoted-document-length-normalisation/

In [52]:
from gensim import corpora, models
tfidf = models.TfidfModel(doc_term_matrix)
corpus_tfidf = tfidf[doc_term_matrix]

In [53]:
#corpus_tfidf[0]##第一个doc里面词的score，可以看到这个score跟doc_term_matrix是不一样的,出现太多的词score小

In [54]:
lda_model_tfidf = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics=25, id2word=dictionary, passes=500)

In [56]:
ldaTfResult=lda_model_tfidf.show_topics(num_topics=25, num_words=20)##可以发现tfidf去掉了很多general的词。

In [57]:
TopicIds=[]
TopicWords={}
for i,j in ldaTfResult:
    TopicWords['topic'+str(i)]=(j.split(" + "))
ldaTfResultDF=pd.DataFrame(data=TopicWords)
ldaTfResultDF   

,topic0,topic1,topic10,topic11,topic12,topic13,topic14,topic15,topic16,topic17,...,topic22,topic23,topic24,topic3,topic4,topic5,topic6,topic7,topic8,topic9
0,"0.012*""gorgeous""","0.020*""stair""","0.016*""overload""","0.008*""circus""","0.013*""scale""","0.012*""highlight""","0.014*""see""","0.019*""august""","0.015*""drove""","0.009*""looked""",...,"0.026*""dull""","0.009*""nowhere""","0.014*""month""","0.008*""see""","0.014*""dirty""","0.011*""visiting""","0.018*""week""","0.013*""everytime""","0.013*""lively""","0.011*""wanting"""
1,"0.010*""appreciate""","0.015*""lived""","0.015*""bustling""","0.008*""treasure""","0.012*""built""","0.011*""soak""","0.014*""night""","0.016*""perfect""","0.012*""artist""","0.007*""absolute""",...,"0.018*""moment""","0.009*""smoker""","0.011*""changing""","0.008*""hotel""","0.012*""woman""","0.009*""late""","0.014*""everybody""","0.010*""pavement""","0.011*""wild""","0.010*""overhead"""
2,"0.010*""america""","0.013*""fall""","0.013*""sensory""","0.008*""island""","0.009*""gotta""","0.008*""replica""","0.013*""place""","0.013*""son""","0.012*""heard""","0.007*""boy""",...,"0.011*""allow""","0.008*""self""","0.009*""depending""","0.008*""people""","0.011*""flyer""","0.008*""impressive""","0.012*""excitement""","0.010*""scooter""","0.010*""prepare""","0.009*""wheel"""
3,"0.009*""bottom""","0.011*""excess""","0.013*""musician""","0.008*""morning""","0.009*""married""","0.008*""ugly""","0.013*""great""","0.013*""compare""","0.011*""disneyland""","0.007*""freak""",...,"0.010*""unlike""","0.008*""expecting""","0.009*""shuttle""","0.008*""walk""","0.010*""porn""","0.007*""complete""","0.011*""location""","0.009*""def""","0.008*""ups""","0.009*""lined"""
4,"0.008*""fab""","0.009*""greatest""","0.010*""daylight""","0.007*""activity""","0.007*""newer""","0.008*""drunken""","0.011*""much""","0.012*""snack""","0.011*""cola""","0.007*""usual""",...,"0.009*""nude""","0.008*""glitzy""","0.008*""dance""","0.007*""time""","0.010*""begging""","0.006*""mostly""","0.011*""distance""","0.007*""tho""","0.008*""depends""","0.009*""electric"""
5,"0.007*""vagrant""","0.009*""coke""","0.010*""crowed""","0.007*""spot""","0.007*""recently""","0.007*""hydrated""","0.011*""fun""","0.011*""christmas""","0.010*""coca""","0.007*""stopped""",...,"0.009*""craziness""","0.008*""hawker""","0.008*""scene""","0.007*""get""","0.009*""men""","0.006*""bigger""","0.010*""unbelievable""","0.007*""marvel""","0.008*""worthwhile""","0.008*""die"""
6,"0.007*""gave""","0.009*""pure""","0.009*""brings""","0.007*""change""","0.007*""ice""","0.007*""erupting""","0.010*""light""","0.010*""degree""","0.010*""amusement""","0.007*""outstanding""",...,"0.009*""leaving""","0.007*""blow""","0.008*""pass""","0.007*""walking""","0.009*""sex""","0.006*""call""","0.009*""stopping""","0.007*""mobility""","0.008*""relaxing""","0.007*""mono"""
7,"0.006*""designer""","0.009*""path""","0.009*""spa""","0.007*""explore""","0.007*""cream""","0.007*""jam""","0.010*""people""","0.009*""sunset""","0.010*""bumper""","0.006*""beggers""",...,"0.008*""ceasars""","0.007*""memorable""","0.007*""share""","0.007*""day""","0.009*""hate""","0.006*""bother""","0.009*""within""","0.007*""conditioning""","0.008*""mood""","0.007*""thanks"""
8,"0.006*""honest""","0.008*""shirt""","0.009*""fat""","0.006*""opportunity""","0.007*""coupon""","0.007*""particular""","0.010*""always""","0.009*""temp""","0.009*""heel""","0.006*""station""",...,"0.008*""figure""","0.006*""modern""","0.007*""care""","0.007*""casino""","0.009*""giving""","0.006*""police""","0.009*""hollywood""","0.007*""grandeur""","0.008*""shoving""","0.007*""ferris"""
9,"0.006*""highway""","0.008*""teen""","0.008*""tuesday""","0.006*""lifetime""","0.006*""sheer""","0.006*""future""","0.010*""love""","0.009*""watched""","0.008*""specially""","0.006*""decoration""",...,"0.008*""anytime""","0.006*""thoroughly""","0.006*""become""","0.006*""place""","0.009*""noisy""","0.006*""blvd""","0.009*""farther""","0.006*""rating""","0.

In [58]:
ldaTfResultDF.to_csv('topic25.csv')

## extract doc topics

In [59]:
print (doc_clean[0])
print (doc_term_matrix[0])

['rated', 'new', 'taken', 'away', 'spark', 'original', 'prepared', 'pant', 'drink', 'bother', 'night', 'club', 'guy', 'pretty', 'sexist', 'priced', 'staff', 'everywhere', 'arrogant', 'care', 'long']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)]


In [60]:
docTopicWeight={}
docTopicsWeightList=lda_model_tfidf.get_document_topics(bow=doc_term_matrix,minimum_probability=0.001)
for i in rawInput.index:
    docTopicWeight[i]=docTopicsWeightList[i]

In [61]:
DocTopicWeightsResult=pd.DataFrame.from_dict(docTopicWeight, orient='index')###very important one for figure out the dict length different issue 
DocTopicWeightsResult.to_csv('DocTopic25WeightsResult.csv')

## typical LDA model(bag of words)

In [63]:
# Creating the object for LDA model using gensim library
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=5, id2word=dictionary, passes=2, workers=2)
#这里workers是用几个cpu的意思，multicore就是多核运算意思

In [64]:
print(lda_model.print_topics(num_topics=5, num_words=10))

[(0, '0.046*"vega" + 0.024*"casino" + 0.023*"la" + 0.022*"people" + 0.018*"hotel" + 0.018*"lot" + 0.017*"fun" + 0.016*"one" + 0.016*"place" + 0.016*"great"'), (1, '0.036*"people" + 0.031*"night" + 0.029*"see" + 0.022*"walk" + 0.021*"light" + 0.020*"walking" + 0.018*"get" + 0.017*"lot" + 0.016*"casino" + 0.014*"day"'), (2, '0.034*"vega" + 0.024*"hotel" + 0.016*"walking" + 0.016*"like" + 0.014*"place" + 0.013*"day" + 0.013*"la" + 0.012*"lot" + 0.012*"see" + 0.012*"one"'), (3, '0.036*"see" + 0.025*"place" + 0.024*"hotel" + 0.022*"walk" + 0.019*"time" + 0.019*"vega" + 0.019*"much" + 0.016*"casino" + 0.013*"show" + 0.013*"walking"'), (4, '0.044*"vega" + 0.032*"time" + 0.030*"people" + 0.020*"day" + 0.018*"hotel" + 0.017*"show" + 0.015*"la" + 0.015*"take" + 0.014*"like" + 0.014*"walking"')]


In [65]:
lda_model.show_topics(num_topics=5, num_words=10)

[(0,
  '0.046*"vega" + 0.024*"casino" + 0.023*"la" + 0.022*"people" + 0.018*"hotel" + 0.018*"lot" + 0.017*"fun" + 0.016*"one" + 0.016*"place" + 0.016*"great"'),
 (1,
  '0.036*"people" + 0.031*"night" + 0.029*"see" + 0.022*"walk" + 0.021*"light" + 0.020*"walking" + 0.018*"get" + 0.017*"lot" + 0.016*"casino" + 0.014*"day"'),
 (2,
  '0.034*"vega" + 0.024*"hotel" + 0.016*"walking" + 0.016*"like" + 0.014*"place" + 0.013*"day" + 0.013*"la" + 0.012*"lot" + 0.012*"see" + 0.012*"one"'),
 (3,
  '0.036*"see" + 0.025*"place" + 0.024*"hotel" + 0.022*"walk" + 0.019*"time" + 0.019*"vega" + 0.019*"much" + 0.016*"casino" + 0.013*"show" + 0.013*"walking"'),
 (4,
  '0.044*"vega" + 0.032*"time" + 0.030*"people" + 0.020*"day" + 0.018*"hotel" + 0.017*"show" + 0.015*"la" + 0.015*"take" + 0.014*"like" + 0.014*"walking"')]

## pyLDAvis  
dictionary/lda/doc_term_matrix可以先保存,然后load的

In [77]:
"""
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
doc_term_matrix = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
"""

"\ndictionary = gensim.corpora.Dictionary.load('dictionary.gensim')\ndoc_term_matrix = pickle.load(open('corpus.pkl', 'rb'))\nlda = gensim.models.ldamodel.LdaModel.load('model5.gensim')\n"

In [146]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda_model_tfidf, doc_term_matrix, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)